In [8]:
import pandas as pd
import numpy as np
import jsonlines
import re
import os
from datetime import datetime
from twython import Twython
import json
import pandas as pd

In [9]:
def flatten_dict(pyobj, keystring=''): 
    if type(pyobj) == dict: 
        keystring = keystring + '_' if keystring else keystring 
        for k in pyobj: 
            yield from flatten_dict(pyobj[k], keystring + str(k)) 
    else: 
        yield keystring, pyobj 

In [10]:
def get_hashtags(hash_list):
    hashtags = ''
    for item in hash_list:
        hashtags = hashtags + ' ' + item['text']
        hashtags = hashtags.strip()
    return hashtags

In [11]:
key_names = ['lang', 'quoted_status_lang', 'created_at', 'id_str', 'full_text', 
             'entities_hashtags', 'entities_urls', 'entities_media',
             'quoted_status_full_text']


In [12]:
my_file = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-00.jsonl'
my_file2 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-01.jsonl'
my_file3 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-02.jsonl'
my_file4 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-03.jsonl'
my_file5 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-09.jsonl'

In [15]:
data = []



for file in [my_file, my_file2, my_file3, my_file4, my_file5]:
    with jsonlines.open(my_file) as f:
        for line in f:
            # keep only if english and not a retweet
            if line['lang'] == 'en':
                data.append(dict(flatten_dict(line)))
    print('Loaded {} records'.format(len(data)))



Loaded 47409 records
Loaded 94818 records
Loaded 142227 records
Loaded 189636 records
Loaded 237045 records


In [14]:
myfile

NameError: name 'myfile' is not defined

In [17]:
## Is there any geotag information in our data?

for i in range(0,1999):
    if (data[i]['geo'] != None):
        print(data[i]['geo'])
    


In [16]:
key_names = data[0].keys()
print(key_names)

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities_hashtags', 'entities_symbols', 'entities_user_mentions', 'entities_urls', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user_id', 'user_id_str', 'user_name', 'user_screen_name', 'user_location', 'user_description', 'user_url', 'user_entities_description_urls', 'user_protected', 'user_followers_count', 'user_friends_count', 'user_listed_count', 'user_created_at', 'user_favourites_count', 'user_utc_offset', 'user_time_zone', 'user_geo_enabled', 'user_verified', 'user_statuses_count', 'user_lang', 'user_contributors_enabled', 'user_is_translator', 'user_is_translation_enabled', 'user_profile_background_color', 'user_profile_background_image_url', 'user_profile_background_image_url_https', 'user_profile_background_tile', 'user_profile_image_url', 'user_profile_image_url_https', 'user_profile_banner

In [18]:
## Will include both files because data included both
db_data = []
for d in data:
    #  make the hashtags into a string with spaces between
    if 'entities_hashtags' in d:
        d['entities_hashtags'] = get_hashtags(d['entities_hashtags'])
    
    db_data.append([])
    for col in key_names:
        db_data[-1].append(d.get(col, ''))
df = pd.DataFrame(db_data, columns=key_names)

In [19]:
df.shape

(237045, 76)

In [22]:
df.user_screen_name[5]

'varunkm6'

In [16]:
data[5]['id']

1223397106116788225

In [21]:
with open('/Volumes/LACIE_SHARE/Software/spark-2.4.6-bin-hadoop2.7/twitter_api_test/twitter_credentials.json', 'r') as file:
        creds = json.load(file)
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])



In [24]:
## Can we get the recent tweets from a user?
covid_tweet_id = data[5]['id']

pre_user_timeline = python_tweets.get_user_timeline(screen_name=data[3]['user_screen_name'], count = 10, max_id = covid_tweet_id)


pre_tweet_dict = {'user': [], 'date': [], 'text':[], 'favorite_count':[], 'tweetid':[]}
for status in pre_user_timeline: ##Store these in the dictionary
            pre_tweet_dict['user'].append(status['user']['screen_name'])
            pre_tweet_dict['favorite_count'].append(status['favorite_count'])
            pre_tweet_dict['text'].append(status['text'])
            pre_tweet_dict['date'].append(status['created_at'])
            pre_tweet_dict['tweetid'].append(status['id'])
            
post_user_timeline = python_tweets.get_user_timeline(screen_name=data[3]['user_screen_name'], count = 10, min_id = covid_tweet_id)


post_tweet_dict = {'user': [], 'date': [], 'text':[], 'favorite_count':[], 'tweetid':[]}
for status in post_user_timeline: ##Store these in the dictionary
            post_tweet_dict['user'].append(status['user']['screen_name'])
            post_tweet_dict['favorite_count'].append(status['favorite_count'])
            post_tweet_dict['text'].append(status['text'])
            post_tweet_dict['date'].append(status['created_at'])
            post_tweet_dict['tweetid'].append(status['id'])
            

print(post_tweet_dict['text'])
print(pre_tweet_dict['text'])

['@Tar0G0 @ppunpunchii อันนี้เป็นสิ่งที่เล่าให้ทุกคนฟังมาตลอด 555555 โตและเรียนมากับรอยลูกกระสุนพวกนี้จริงๆ อยู่ยังไงอยู่ยังงั้น ชอบค่ะ', '@underseaNavy 500!. I googled it! 😂', '@oakpc2002 @Violet16975590 Why don’t we just throw 1976 and 1991 as well. It’s always freaking the same thing. People vs military.', '@Violet16975590 It’s not the people against the police here. It’s the people against military. Police saw people go… https://t.co/6iv342WTdt', '@Violet16975590 The police joined protesters against the military. The coup used the Army, tanks, snipers and live… https://t.co/9TUjOj8CAG', '@nn07783169 @PollyPloy24 @lovenoey101 ได้แต่ขอให้เวรกรรมตามทันไม่ทางใดก็ทางนึงค่ะ ตอนแรกเห็นเงียบไปเราก็นึกไปในทาง… https://t.co/tESHgBgb8a', '@ThaiPBS เคยอ่านงานวิจัยผู้ติดเชื้อของจีน แต่จำนวนเคสที่ศึกษามีไม่มากค่ะตอนนั้น น่าจะไม่กี่ร้อยเคส เค้าบอกว่าไม่สาม… https://t.co/IVydn4WV5J', '@thidakarn ถ้าคนไหนที่กินฟ้าทะลายโจรต่อเนื่องแล้วเป็นผื่นคัน ลองหยุดกินดูซักพักนะคะ เป็นมาแล้วค่ะผื่นคันเพราะว่ากิ

In [24]:
tweet_dict.keys()

{'user': ['Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita',
  'Linniethepita'],
 'date': ['Sat Jul 11 08:35:19 +0000 2020',
  'Sun May 31 09:31:11 +0000 2020',
  'Sun May 31 06:39:45 +0000 2020',
  'Sun May 31 06:30:53 +0000 2020',
  'Wed Apr 29 06:29:32 +0000 2020',
  'Mon Apr 13 17:37:56 +0000 2020',
  'Sun Mar 29 05:58:49 +0000 2020',
  'Wed Mar 18 12:01:31 +0000 2020',
  'Wed Mar 18 11:50:22 +0000 2020',
  'Wed Mar 18 02:14:04 +0000 2020'],
 'text': ['@underseaNavy 500!. I googled it! 😂',
  '@oakpc2002 @Violet16975590 Why don’t we just throw 1976 and 1991 as well. It’s always freaking the same thing. People vs military.',
  '@Violet16975590 It’s not the people against the police here. It’s the people against military. Police saw people go… https://t.co/6iv342WTdt',
  '@Violet16975590 The police joined protesters against the military. The coup used the Army, tanks, 

In [25]:
### How many of the people they follow are mutuals?
current_user_name = data[5]["user_screen_name"]

follower_list = []

next_cursor = -1

while (next_cursor):
    user_follower_list = python_tweets.get_followers_list(screen_name = current_user_name, count = 200, cursor = next_cursor)
    next_cursor = user_follower_list['next_cursor']
    print(next_cursor)
    for user in user_follower_list['users']:
        follower_list.append(user['screen_name'])





0


In [26]:
following_list = []

next_cursor = -1

while(next_cursor): ## will be zero when we get to the end
    user_following_list = python_tweets.get_friends_list(screen_name = current_user_name, count = 200, cursor = next_cursor)
    next_cursor = user_following_list['next_cursor']
    print(next_cursor)
    for user in user_following_list['users']:
        following_list.append(user['screen_name'])


    

1650667663980023823
0


In [27]:
len(following_list)


298

In [28]:
len(follower_list)


43

In [29]:
mutuals_list = list(set(following_list) & set(follower_list))

In [30]:
len(mutuals_list)

11

In [72]:
##Now, let's put it together to get the mutuals for any user. 
## will send too many requests so don't run that

list_of_followers = [[]]

list_of_friends = [[]]

list_of_screen_names = []

list_of_mutuals = [[]]

for i in range(0, 10):
    current_user_name = data[i]["user_screen_name"]

    follower_list = []

    next_cursor = -1

    while (next_cursor):
        user_follower_list = python_tweets.get_followers_list(screen_name = current_user_name, count = 200, cursor = next_cursor)
        next_cursor = user_follower_list['next_cursor']
        print(next_cursor)
        for user in user_follower_list['users']:
            follower_list.append(user['screen_name'])
    
    list_of_followers.append(follower_list)
    
    following_list = []

    next_cursor = -1

    while(next_cursor): ## will be zero when we get to the end
        user_following_list = python_tweets.get_friends_list(screen_name = current_user_name, count = 200, cursor = next_cursor)
        next_cursor = user_following_list['next_cursor']
        print(next_cursor)
        for user in user_following_list['users']:
            following_list.append(user['screen_name'])
    list_of_friends.append(following_list)
    
    mutuals_list = list(set(following_list) & set(follower_list))
    
    list_of_mutuals.append(mutuals_list)
    
    list_of_screen_names.append(current_user_name)
    



1668929260011288767
1658989162346664684
1632528552194136112
1619123260177824017
1610078298643690827
1602422611077069778
1596915621987771019
1592973248723863718
1589814472980413312
1585881171025853909
1581511163261876080
1575306043346467657
1569296179055611404
1563302035620236940
1555722992947794091
1545823433070593990
1537402789045084525
1520021812275441964
1507409775911117732
1495359380025447014
1484106602810104571
1473803821521253732
0
1666034306272620065
1665325217324339841
1621751744586253567
1612294900955752870
1605960554288654242
1598894157086953131
1594479332164682867
1591562674157968853
1588035247000454545
1584146148742731974
1579241608519710375
1573523009591818722
1567025703682131986
1561205368662386805
1552593423412066354
1542535321222548891
1532961044406242668
1508669622756338640
1497054938341802094
1485007464409005189
1473966069764853273
0
1673663916083994556
1668688081686312310
1665503871379902372
1661561782746322247
1657393896191472106
1654942763252902853


TwythonRateLimitError: Twitter API returned a 429 (Too Many Requests), Rate limit exceeded

In [34]:
## Finding which users made the most tweets



user_list = []
number_list = []

for i in range(0, df.shape[0]):
    if(df.user_screen_name[i] in user_list):
        number_list[user_list.index(df.user_screen_name[i])] += 1
    else:
        number_list.append(1)
        user_list.append(df.user_screen_name[i])
    if (i % 1000 == 0):
        print(i)
        
        
tweets_by_user = pd.DataFrame({'user': user_list, 'number': number_list})



tweets_by_user.sort_values(by='number', axis=0, ascending=False, 
                           inplace=True)

print(tweets_by_user[0:20])





KeyboardInterrupt: 

In [38]:
print(tweets_by_user.shape)
tweets_by_user = pd.DataFrame({'user': user_list, 'number': number_list})
tweets_by_user.sort_values(by='number', axis=0, ascending=False, 
                           inplace=True)
print(tweets_by_user[0:20])

(36875, 2)
                  user  number
12839   avenger_naren2     132
10519    YanHongkonger     113
1413   BaronianConsult     111
1485         QuitCheng     105
808          viriyabot      98
523         adsertoris      96
117        sinnieloves      93
18914          SavehkH      87
3509    NikolayShubin1      87
10395          Puixxx1      76
19054         xsutingx      76
232      Iceholiccream      76
63          VitaminKLo      72
10300     JoJo53859628      68
5164     Gabriellayes2      64
11002       COVID19USA      63
12669    realllllcindy      60
10366   DavidMikeHarry      57
12036    john1966olsen      54
12612      BCquakehelp      54


In [40]:
i

121473

In [39]:
user_list = ['me', 'you', 'her']
number_list = [3, 5, 6]
number_list[user_list.index('her')] += 2


print(number_list[user_list.index('her')])

print('me' in user_list)
print('her' in user_list)




8
True
True


In [39]:
df.source[0]




'<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>'

In [42]:
## How many users are tweeting from different platforms
platform_list = []
number_list = []

for i in range(0, 1000):
    if(df.source[i] in platform_list):
        number_list[platform_list.index(df.source[i])] += 1
    else:
        number_list.append(1)
        platform_list.append(df.source[i])
        
        
tweets_by_source = pd.DataFrame({'platform': platform_list, 'number': number_list})



tweets_by_user.sort_values(by='number', axis=0, ascending=False, 
                          inplace=True)

print(tweets_by_source)



                                             platform  number
0   <a href="http://twitter.com/download/android" ...     324
1   <a href="http://twitter.com/download/iphone" r...     408
2   <a href="https://mobile.twitter.com" rel="nofo...     157
3   <a href="http://twitter.com/#!/download/ipad" ...      36
4   <a href="http://www.grouptweet.com" rel="nofol...       1
5   <a href="http://english.yonhapnews.co.kr" rel=...       1
6   <a href="http://tapbots.com/tweetbot" rel="nof...       3
7   <a href="http://twitter.com" rel="nofollow">Tw...      17
8   <a href="http://thin.air" rel="nofollow">gener...       1
9   <a href="https://about.twitter.com/products/tw...       7
10  <a href="http://www.tweetcaster.com" rel="nofo...       1
11  <a href="http://www.samruston.co.uk" rel="nofo...       1
12  <a href="https://ifttt.com" rel="nofollow">IFT...       1
13  <a href="https://thetop10news.com/" rel="nofol...       1
14  <a href="https://malco.io" rel="nofollow">epi_...       1
15  <a h

In [43]:
tweets_by_source.sort_values(by='number', axis=0, ascending=False, 
                          inplace=True)

print(tweets_by_source)

                                             platform  number
1   <a href="http://twitter.com/download/iphone" r...     408
0   <a href="http://twitter.com/download/android" ...     324
2   <a href="https://mobile.twitter.com" rel="nofo...     157
3   <a href="http://twitter.com/#!/download/ipad" ...      36
7   <a href="http://twitter.com" rel="nofollow">Tw...      17
15  <a href="https://dlvrit.com/" rel="nofollow">d...      13
9   <a href="https://about.twitter.com/products/tw...       7
27  <a href="http://publicize.wp.com/" rel="nofoll...       4
35  <a href="http://www.powerapps.com" rel="nofoll...       4
6   <a href="http://tapbots.com/tweetbot" rel="nof...       3
29  <a href="http://www.socialnewsdesk.com" rel="n...       2
8   <a href="http://thin.air" rel="nofollow">gener...       1
23  <a href="http://bitcoinagile.com" rel="nofollo...       1
34  <a href="https://twitter.com/ViRiYaBOT" rel="n...       1
33  <a href="http://www.tobeannounced.com" rel="no...       1
32  <a h